Mean Reversion Trading Algorithm
 - A mean reversion trading strategy is like spotting when a stock or investment strays really far from its usual value.
 - The main thought behind this is that, eventually, it tends to return closer to its usual value.
 - There are various methods to make this strategy work, like using tools such as linear regression or moving averages.
 - The key question is: How much of a deviation is considered too much before it's likely to come back?


In [1]:
import pandas as pd
import pandas_datareader as pdr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['figure.figsize'] = 8,6
import seaborn as sb
sb.set()

In [ ]:
gld = pd.DataFrame(pdr.get_data_yahoo('GLD')['Close'])

In [ ]:
gld.head()

In [ ]:
ma = 21
gld['returns'] = np.log(gld["Close"]).diff()
gld['ma'] = gld['Close'].rolling(ma).mean()
gld['ratio'] = gld['Close'] / gld['ma']

In [ ]:
gld['ratio'].describe()

In [ ]:
percentiles = [5, 10, 50, 90, 95]
p = np.percentile(gld['ratio'].dropna(), percentiles)

In [ ]:
gld['ratio'].dropna().plot(legend = True)
plt.axhline(p[0], c= (.5,.5,.5), ls='--')
plt.axhline(p[2], c= (.5,.5,.5), ls='--')
plt.axhline(p[-1], c= (.5,.5,.5), ls='--');

In [ ]:
short = p[-1]
long = p[0]
gld['position'] = np.where(gld.ratio > short, -1, np.nan)
gld['position'] = np.where(gld.ratio < long, 1, gld['position'])
gld['position'] = gld['position'].ffill()

In [ ]:
gld.position.dropna().plot()

In [ ]:
gld['strat_return'] = gld['returns'] * gld['position'].shift()

In [ ]:
plt.plot(np.exp(gld['returns'].dropna()).cumprod(), label='Buy/Hold')
plt.plot(np.exp(gld['strat_return'].dropna()).cumprod(), label='Strategy')
plt.legend();

In [ ]:
print(np.exp(gld['returns'].dropna()).cumprod()[-1] -1)
print(np.exp(gld['strat_return'].dropna()).cumprod()[-1] - 1)